In [1]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/Development/
  inflating: data/Development/dev_definitions.txt  
  inflating: data/Development/dev_examples.txt  
  inflating: data/Development/dev_hypernyms.txt  
  inflating: data/Development/dev_labels.txt  
  inflating: data/README.txt         
   creating: data/Test/
  inflating: data/Test/test_definitions.txt  
  inflating: data/Test/test_examples.txt  
  inflating: data/Test/test_hypernyms.txt  
   creating: data/Training/
  inflating: data/Training/train_definitions.txt  
  inflating: data/Training/train_examples.txt  
  inflating: data/Training/train_hypernyms.txt  
  inflating: data/Training/train_labels.txt  


In [2]:
!pip install tensorflow_text
!pip install transformers

     |████████████████████████████████| 4.9 MB 5.4 MB/s 
     |████████████████████████████████| 462 kB 46.0 MB/s 
     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 41.1 MB/s 
     |████████████████████████████████| 596 kB 45.5 MB/s 
     |████████████████████████████████| 6.5 MB 39.2 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import re
import os
import errno
import pandas as pd

!rm -rf processed_data

try:
    os.makedirs("processed_data")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [2]:
def decontracted(phrase):

    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can't", "can not", phrase)
    phrase = re.sub(r"gonna", "going to", phrase)
    phrase = re.sub(r"wanna", "want to", phrase)

    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\.+", " ", phrase)
    phrase = re.sub(r"[^A-Za-z$]", " ", phrase)
    phrase = re.sub(r" +", " ", phrase)

    return phrase

In [3]:
def get_df(pref, testing=False):

    dataset = []
    with open(pref + "_" + "examples.txt") as examples:
        for example in examples:
            example = example[:-1].lower().split("\t")
            if example[0] != decontracted(example[0]):
                text = re.sub(example[0], "\$", example[2])
                text = decontracted(text)
                text = re.sub(r"\$", example[0], text)
                example[2] = text
            else:
                example[2] = decontracted(example[2])
            dataset.append(example)

    with open(pref + "_" + "hypernyms.txt") as hypernyms:
        for index, line in enumerate(hypernyms):
            line = line[:-1].lower().split("\t")
            line = " ; ".join(line)
            line = re.sub(r'_', ' ', line)
            dataset[index].append(line)

    with open(pref + "_" + "definitions.txt") as definitions:
        for index, line in enumerate(definitions):
            line = line[:-1].lower().split(";")
            for i in range(len(line)):
                line[i] = line[i].strip()
                if dataset[index][0] != decontracted(dataset[index][0]):
                    text = re.sub(dataset[index][0], "\$", line[i])
                    text = decontracted(text)
                    text = re.sub(r"\$", dataset[index][0], text)
                    line[i] = text
                else:
                    line[i] = decontracted(line[i])
            line = " ; ".join(line)
            dataset[index].append(line)

    cols = ["target", "position", "sentence", "hypernym", "definition"]

    if not testing:
        cols.append("label")
        with open(pref + "_" + "labels.txt") as labels:
            for index, line in enumerate(labels):
                line = line[:-1]
                dataset[index].append(line)

    df = pd.DataFrame(dataset, columns=cols)

    return df

In [4]:
dev_df = get_df("data/Development/dev")
train_df = get_df("data/Training/train")
test_df = get_df("data/Test/test", True)

dev_df.to_csv("processed_data/dev.csv", index=None)
train_df.to_csv("processed_data/train.csv", index=None)
test_df.to_csv("processed_data/test.csv", index=None)

In [5]:
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import pandas as pd

from keras import backend as K

import transformers
transformers.logging.set_verbosity_error()

from transformers import BertTokenizer
from tensorflow.keras.optimizers import Adam

In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

transformers.logging.set_verbosity_error()

def extract_data(filename, test_data = False):

    df = pd.read_csv(filename)
    X = {'input_type_ids':[],'input_word_ids':[],'input_mask':[], 'mask1': [], 'mask2': []}
    Y = []

    for _, row in df.iterrows():
        word = row['target']
        sentence = row['sentence']
        hypernyms = row['hypernym']
        definitions = row['definition']

        if not test_data:
          if row['label'] == 'T':
              label = 1.0
          else:
              label = 0.0

        desc = [definitions]

        if isinstance(hypernyms, str):
          desc.append(hypernyms)

        desc = ' ; '.join(desc)

        sentence = sentence.replace(word.lower(), '$ ' + word + ' $')

        x = tokenizer(sentence, desc, max_length = 128, padding='max_length', truncation = True)

        sep1 = x['input_ids'].index(102)
        sep2 = x['input_ids'].index(102, sep1 + 1)

        x['mask1'] = np.zeros((128))
        x['mask2'] = np.zeros((128))

        for i in range(0, sep1):
            x['mask1'][i] = 1.0
        
        for i in range(sep1, sep2):
            x['mask2'][i] = 1.0

        x['token_type_ids'][row['position'] + 1] = 1

        X['input_type_ids'].append(x['token_type_ids'])
        X['input_word_ids'].append(x['input_ids'])
        X['input_mask'].append(x['attention_mask'])
        X['mask1'].append(x['mask1'])
        X['mask2'].append(x['mask2'])

        if not test_data:
          Y.append(label)     

    X['input_type_ids'] = np.array(X['input_type_ids'], dtype=np.float32)
    X['input_word_ids'] = np.array(X['input_word_ids'], dtype=np.float32)
    X['input_mask'] = np.array(X['input_mask'], dtype=np.float32)

    X['mask1'] = np.array(X['mask1'], dtype=np.float32)
    X['mask2'] = np.array(X['mask2'], dtype=np.float32)

    Y = np.array(Y)

    return X, Y

X,Y = extract_data('processed_data/train.csv')
Xval, Yval = extract_data('processed_data/dev.csv')

In [7]:
# Input layers -----------------------------

input_type_ids = tf.keras.layers.Input(shape = (128), dtype = tf.int32, name = "input_type_ids")
input_word_ids = tf.keras.layers.Input(shape = (128), dtype = tf.int32, name = "input_word_ids")
input_mask = tf.keras.layers.Input(shape = (128), dtype = tf.int32, name = "input_mask")

input_sent_mask = tf.keras.layers.Input(shape=(128), dtype = bool, name = "input_sent_mask")
input_desc_mask = tf.keras.layers.Input(shape=(128), dtype = bool, name = "input_desc_mask")

# -------------------------------------------


# Encoder -----------------------------------

encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", name = "bert-encoder", trainable = True)
pooled_output, sequence_output = encoder([input_word_ids, input_mask, input_type_ids])

# -------------------------------------------


# Decoder -----------------------------------

def custom_decoder(tensors):
    
    sequence_output = tensors[0]
    sent_mask = tensors[1]
    desc_mask = tensors[2]

    sent_output = tf.reduce_mean(tf.ragged.boolean_mask(sequence_output, mask = sent_mask), axis = 1)

    desc_output = tf.reduce_mean(tf.ragged.boolean_mask(sequence_output, mask = desc_mask), axis = 1)

    return sent_output, desc_output

decoder = tf.keras.layers.Lambda(custom_decoder, name = "decoder")
sent_output, desc_output  = decoder([sequence_output, input_sent_mask, input_desc_mask])

concat = tf.keras.layers.Concatenate(name = "concatenate")([sent_output, desc_output, pooled_output])
dropout = tf.keras.layers.Dropout(0.3, name = "dropout")(concat)
output = tf.keras.layers.Dense(1, activation = 'sigmoid', name = "output")(dropout)

# -------------------------------------------

# Encoder-Decoder Model ---------------------

model = tf.keras.models.Model(
      inputs = [
        input_word_ids,
        input_mask,
        input_type_ids,
        input_sent_mask,
        input_desc_mask], 
      outputs = output)

model.compile(optimizer=Adam(learning_rate = 0.00005), loss="binary_crossentropy", metrics=["accuracy"])

# -------------------------------------------

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert-encoder (KerasLayer)      [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 128, 768)]                'input_mask[0][0]',         

In [8]:
class StopOnPoint(tf.keras.callbacks.Callback):
    def __init__(self):
        super(StopOnPoint, self).__init__()

    def on_epoch_end(self, epoch, logs = None): 
        self.model.save('model.' + str(epoch) + '.h5')

In [9]:
history = model.fit([X['input_word_ids'], X['input_mask'], X['input_type_ids'], X['mask1'], X['mask2']], Y, 
          epochs = 3, batch_size = 16, callbacks=[StopOnPoint()],
          validation_data = ([Xval['input_word_ids'], Xval['input_mask'], Xval['input_type_ids'], Xval['mask1'], Xval['mask2']], Yval)
          )

Epoch 1/10
134/134 [==============================] - 141s 904ms/step - loss: 0.6433 - accuracy: 0.6359 - val_loss: 0.6198 - val_accuracy: 0.6735
Epoch 2/10
134/134 [==============================] - 120s 892ms/step - loss: 0.3831 - accuracy: 0.8484 - val_loss: 0.7172 - val_accuracy: 0.6710
Epoch 3/10
134/134 [==============================] - 119s 892ms/step - loss: 0.1263 - accuracy: 0.9593 - val_loss: 0.8171 - val_accuracy: 0.7147
Epoch 4/10
 26/134 [====>.........................] - ETA: 1:26 - loss: 0.0810 - accuracy: 0.9712

KeyboardInterrupt: ignored

In [10]:
epoch = 2
pretrained_model = tf.keras.models.load_model('model.' + str(epoch) + '.h5', custom_objects={'KerasLayer': hub.KerasLayer})

In [11]:
Xtest, Ytest = extract_data('processed_data/test.csv', True)
y_predicted = pretrained_model.predict([Xtest['input_word_ids'], Xtest['input_mask'], Xtest['input_type_ids'], Xtest['mask1'], Xtest['mask2']])
y_predicted = y_predicted.flatten()

In [12]:
file = open('all_output.txt', 'w')
for val in y_predicted:
  if val < 0.5:
    print('F',file=file)
  else:
    print('T',file=file)
file.close()

In [92]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
!cp "model.2.h5" "gdrive/MyDrive/WSD/BERT-BEST2.h5"